In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#instala o openyxl para fazer o handling do arquivo de excel
!pip install openpyxl

/kaggle/input/supersession/SuperSession.xlsx
     |████████████████████████████████| 242 kB 294 kB/s            


In [3]:
Error_Log = pd.DataFrame()
Error_Log

""


In [4]:
# Seção para obter o arquivo input do SuperSession e armazená-lo em um DataFrame pandas

original_ss_address = "../input/supersession/SuperSession.xlsx"
ss = pd.read_excel(original_ss_address, engine="openpyxl")

In [5]:
# Seção para verificar se existem as cadeias globais na lista (foi feito um Workaround para selecionar TODAS as cadeias do arquivo, logo todos serão selecionados)

ss_profile_global = pd.DataFrame(ss["GROUP_NUMBER"].unique(), columns = ['GROUP_NUMBER'])
#ss_profile_global = pd.DataFrame(["C28644_010", "C29362_010"], columns = ['GROUP_NUMBER'])

ss = ss[ss.GROUP_NUMBER.isin(ss_profile_global.GROUP_NUMBER)]

In [6]:
ss['A_B'] = ss["PREC_PRODUCT"] +"_"+  ss["SUCC_PRODUCT"]
ss['B_A'] = ss["SUCC_PRODUCT"] +"_"+  ss["PREC_PRODUCT"]

In [7]:
# Seção para retirar a duplicidade onde um Predecessor possui mais de um Sucessor
ss_1PREC_NSUCC = pd.pivot_table(ss, values='SUCC_PRODUCT', index='PREC_PRODUCT',
               aggfunc='count')

ss_1PREC_NSUCC = ss_1PREC_NSUCC.loc[ss_1PREC_NSUCC['SUCC_PRODUCT'] > 1]

ss = ss[-ss.PREC_PRODUCT.isin(ss_1PREC_NSUCC.index)]

In [8]:
# Seção para retirar a duplicidade onde um Sucessor possui mais de um Predecessor quando a cadeia é do tipo Duas Vias

ss_YY = ss.loc[ss["TIPO_INTERC"] == "I"]

ss_1SUCC_NPREC_YY = pd.pivot_table(ss_YY, values='PREC_PRODUCT', index='SUCC_PRODUCT',
               aggfunc='count')

ss_1SUCC_NPREC_YY = ss_1SUCC_NPREC_YY.loc[ss_1SUCC_NPREC_YY['PREC_PRODUCT'] > 1]

ss = ss[-ss.SUCC_PRODUCT.isin(ss_1SUCC_NPREC_YY.index)]

In [9]:
roda_inversao = False
if roda_inversao:
    ss_inversion_A_B = pd.DataFrame()
    ss_inversion_B_A = pd.DataFrame()
    ss_inversion_A_B["A_B"] = ss["PREC_PRODUCT"] +"_"+  ss["SUCC_PRODUCT"]
    ss_inversion_B_A["B_A"] = ss["SUCC_PRODUCT"] +"_"+  ss["PREC_PRODUCT"]
    ss_inversion_A_B = ss_inversion_A_B[ss_inversion_A_B.A_B.isin(ss_inversion_B_A.B_A)]
    ss_inversion_B_A = ss_inversion_B_A[ss_inversion_B_A.B_A.isin(ss_inversion_A_B.A_B)]
    ss = ss[-ss.A_B.isin(ss_inversion_A_B.A_B)]
    ss = ss[-ss.B_A.isin(ss_inversion_B_A.B_A)]
    
    print("rodou inversão de cadeia...")

In [10]:
# Armazena o arquivo limpo do SuperSession (tratado acima) em um arquivo xlsx para análise
ss.to_excel("SuperSession_Clean.xlsx")

In [11]:
ss.head() #exibe uma amostra do DataFrame (head: 5 primeiras linhas)

,GROUP_NUMBER,PREC_PRODUCT,SUCC_PRODUCT,TIPO_INTERC,A_B,B_A
0,BF2LCHD,BF2D4LCHD,BF2DK2LCHD,I,BF2D4LCHD_BF2DK2LCHD,BF2DK2LCHD_BF2D4LCHD
1,C00002_010,16100ZF6W20,16100ZF6W21,F,16100ZF6W20_16100ZF6W21,16100ZF6W21_16100ZF6W20
2,C00003_010,90012881000,90012882000,I,90012881000_90012882000,90012882000_90012881000
3,C00004_010,44301098000,44301107000,F,44301098000_44301107000,44301107000_44301098000
4,C00004_010,44301107000,44301098010,F,44301107000_44301098010,44301098010_44301107000


In [58]:
#encontra todos os predecessores e sucessores únicos
all_material_Predecessor = pd.DataFrame(ss["PREC_PRODUCT"].unique(), columns = ['PREC_PRODUCT'])
all_material_Successor = pd.DataFrame(ss["SUCC_PRODUCT"].unique(), columns = ['SUCC_PRODUCT'])

#renomeia as colunas de PREC_PRODUCT e SUCC_PRODUCT para Materiais
all_material_Predecessor.rename(columns={"PREC_PRODUCT": "Materiais"}, inplace=True)
all_material_Successor.rename(columns={"SUCC_PRODUCT": "Materiais"}, inplace=True)

#cria um DataFrame para armazenar todos os PNs únicos, sejam eles Predecessores ou Sucessores
all_material = pd.DataFrame()
all_material = all_material.append(all_material_Predecessor)
all_material = all_material.append(all_material_Successor)
all_material = pd.DataFrame(all_material["Materiais"].unique(), columns = ['Materiais'])

In [64]:
#Declara a Função Material_Predecessor() que é uma função auxiliar utilizada em Material_Priorities()
def Material_Predecessor(Material):
    
    if int(all_material_Successor.loc[all_material_Successor["Materiais"] == Material ].count()) >= 1: #Se Material é sucessor de uma linha
        if ss.loc[(ss["SUCC_PRODUCT"] == Material) & (ss["TIPO_INTERC"] == "I")].PREC_PRODUCT.count() == 1:
            var_return = ss.loc[(ss["SUCC_PRODUCT"] == Material) & (ss["TIPO_INTERC"] == "I")].iat[0,1] #localiza e TRAZ A COLUNA 1 DO PREDECESSOR
        else:
            var_return = Material
    elif int(all_material_Predecessor.loc[all_material_Predecessor["Materiais"] == Material].count()) >= 1: #Se Material é sucessor de uma linha
        var_return = Material 
    else:
        var_return = "0"  
    return var_return

In [65]:
#Declara a Função Material_Successor() que é uma função auxiliar utilizada em Material_Priorities()
def Material_Successor(Material):
    if int(all_material_Predecessor.loc[all_material_Predecessor["Materiais"] == Material].count()) >= 1:#Se Material é sucessor de uma linha
        var_return = ss.loc[(ss["PREC_PRODUCT"] == Material)].iat[0,2] # Então retornar Material_Sucessor
      
    elif int(all_material_Successor.loc[all_material_Successor["Materiais"] == Material ].count()) >= 1: # Senão se Material é sucessor de uma linha
        var_return = Material # Então retornar Material

    else:
        var_return = "0" # Senão retornar “0”
        
    return var_return

In [66]:
#Declara a Função Material_Priority1() que é uma função auxiliar utilizada em Material_Priorities()
def Material_Priority1(Material):
    Material_Anterior = Material_Predecessor(Material)
    if Material_Anterior == "0":
        var_return = Material_Anterior
        #print("var_return, ", var_return)
    else:
        Prioridade_1 = Material_Predecessor(Material_Anterior)
        while not (Prioridade_1 == Material_Anterior):
            
            Material_Anterior = Prioridade_1
            
            Prioridade_1 = Material_Predecessor(Material_Anterior)
            
        var_return = Prioridade_1
    return var_return

In [67]:
# Função Material_Priorities
# Função principal que tem como por objetivo verificar todos os predecessores e sucessores de um determinado material
# oferecendo uma lista de prioridades a ser buscada 

def Material_Priorities(Material):
    Priority = pd.DataFrame()
    Priority_1 = Material_Priority1(Material)
    if Priority_1 == "0":
        Priority
    else:   
        i = 0
        
        Priority = Priority.append({'Material': Material,
                                   'Priority': i+1,
                                   'Alt_Material': Priority_1}, ignore_index=True)
        Next_Material = Material_Successor(Priority_1)
        while not Next_Material == Priority.Alt_Material[i]:
            i += 1
            
            for j in range(0, i):
                if Next_Material == Priority.Alt_Material[j]:
                    global Error_Log
                    Error_Log = Error_Log.append({'Program': "Material_Priorities",
                                   'Error': "Cadeia ciclica: " + Material}, ignore_index=True)
            
                    return Priority

            Priority = Priority.append({'Material': Material,
                                   'Priority': i+1,
                                   'Alt_Material': Next_Material}, ignore_index=True)
            
            Next_Material = Material_Successor(Next_Material)    

    return Priority

In [ ]:
all_ss_prioritized = pd.DataFrame()
for i, row in all_material.iterrows():
    nrows = len(all_material.index)
    if i % 10 == 0:
        print( i, end=" ", flush=True)
    all_ss_prioritized = all_ss_prioritized.append(Material_Priorities(all_material.Materiais[i]), ignore_index=True)

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 2

In [ ]:
all_ss_prioritized.to_excel("SuperSession_Prioritized.xlsx")
Error_Log.to_excel("Error_Log.xlsx")